# Question-2



###  What can we do?

Divided all clients with a dp1 Score into 10 score groups (deciles).  
For each group, we calculated:
- Average arrears balance (how much they owe)  
- % of clients with a mobile number  
- % with a valid email

###  What we Can Look in the Table:

- Do higher score deciles have better contactability?  
  (e.g., are email/mobile rates going up from Decile 1 → 10?)
- Do lower scores link to higher debt?  
  If Decile 1 has the highest arrears, it may show high risk
- Where’s the sweet spot?  
  If a middle or high decile has high contact and moderate arrears, it could be a good target for soft debt strategies

---



##  dp1 Score Group Analysis

#### For DP1 Score
- 234 clients had missing scores  
- Only 766 clients were available for decile segmentation

Around 23% of clients had no dp1 score, likely due to no match or insufficient credit data.  
I excluded them from decile analysis but kept them in mind as a potential high-risk group.  
In production, they could be handled separately or flagged for tracing.

In [2]:
import pandas as pd


df = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Modified/cleaned_connected_data_with_zones.csv")

# Step 1: Remove rows with missing dp1 Score
dp1_df = df[df['dp1 Score'].notnull()].copy()

# Step 2: Create deciles based on dp1 Score
dp1_df['dp1 Score Decile'] = pd.qcut(dp1_df['dp1 Score'], 10, labels=[f'Decile {i}' for i in range(1, 11)])

# Step 3: Clean up contact flags
dp1_df['Mobile Flag'] = dp1_df['Mobile Flag'].str.strip().str.upper()
dp1_df['Email Flag'] = dp1_df['Email Flag'].str.strip().str.upper()

# Step 4: Group by decile and calculate key metrics
summary = dp1_df.groupby('dp1 Score Decile').agg(
    Total_Clients=('UID', 'count'),
    Avg_Arrears=('Arrears Balance', 'mean'),
    Mobile_Y=('Mobile Flag', lambda x: (x == 'Y').sum()),
    Email_Y=('Email Flag', lambda x: (x == 'Y').sum())
).reset_index()

# Step 5: Calculate percentage availability
summary['% Mobile Available'] = (summary['Mobile_Y'] / summary['Total_Clients'] * 100).round(1)
summary['% Email Available'] = (summary['Email_Y'] / summary['Total_Clients'] * 100).round(1)

# Step 6: Select and rename final columns
summary = summary[['dp1 Score Decile', 'Total_Clients', 'Avg_Arrears', '% Mobile Available', '% Email Available']]

# Optional: Format float display (optional)
pd.options.display.float_format = '{:,.2f}'.format

# Step 7: result
print("\nDP1 SCORE DECILE ANALYSIS:\n")
print(summary.to_string(index=False))



DP1 SCORE DECILE ANALYSIS:

dp1 Score Decile  Total_Clients  Avg_Arrears  % Mobile Available  % Email Available
        Decile 1             77     1,899.64               26.00              27.30
        Decile 2             83     3,698.15                8.40              19.30
        Decile 3             71     2,674.10               16.90              36.60
        Decile 4             80     2,168.44               21.20              50.00
        Decile 5             73     1,756.27               23.30              46.60
        Decile 6             78     1,050.95               50.00              30.80
        Decile 7             83       283.27               65.10              26.50
        Decile 8             69       198.60               60.90              31.90
        Decile 9             91       197.86               69.20              18.70
       Decile 10             61        55.25               62.30              21.30


I included clients without a dp1 Score as a separate segment, **‘No Score (Unmatched)’**.  
They could represent high-risk or untraceable clients.  
This gives the business full visibility, and prevents silent blind spots in outreach strategy.

In [ ]:
import pandas as pd


df = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Modified/cleaned_connected_data_with_zones.csv")

# Step 1: Create dp1 Score Group (deciles + unmatched)
# Create decile labels for non-null dp1 Scores
df['dp1 Score Group'] = pd.qcut(df[df['dp1 Score'].notnull()]['dp1 Score'], 10, labels=[f'Decile {i}' for i in range(1, 11)])
df['dp1 Score Group'] = df['dp1 Score Group'].astype(str)

# Assign 'No Score' to rows with missing dp1 Score
df.loc[df['dp1 Score'].isnull(), 'dp1 Score Group'] = 'No Score (Unmatched)'

# Step 2: Clean mobile/email flags
df['Mobile Flag'] = df['Mobile Flag'].str.strip().str.upper()
df['Email Flag'] = df['Email Flag'].str.strip().str.upper()

# Step 3: Group by dp1 Score Group and summarize metrics
summary = df.groupby('dp1 Score Group').agg(
    Total_Clients=('UID', 'count'),
    Avg_Arrears=('Arrears Balance', 'mean'),
    Mobile_Y=('Mobile Flag', lambda x: (x == 'Y').sum()),
    Email_Y=('Email Flag', lambda x: (x == 'Y').sum())
).reset_index()

# Step 4: Calculate contactability percentages
summary['% Mobile Available'] = (summary['Mobile_Y'] / summary['Total_Clients'] * 100).round(1)
summary['% Email Available'] = (summary['Email_Y'] / summary['Total_Clients'] * 100).round(1)

# Step 5: Reorder groups so 'No Score' comes last
ordered_labels = [f'Decile {i}' for i in range(1, 11)] + ['No Score (Unmatched)']
summary['dp1 Score Group'] = pd.Categorical(summary['dp1 Score Group'], categories=ordered_labels, ordered=True)
summary = summary.sort_values('dp1 Score Group')

# Step 6: Final selection of columns
summary = summary[['dp1 Score Group', 'Total_Clients', 'Avg_Arrears', '% Mobile Available', '% Email Available']]

# Step 7: output
pd.options.display.float_format = '{:,.2f}'.format
print("\n dp1 Score Group Analysis (Including Missing Group):\n")
print(summary.to_string(index=False))



 dp1 Score Group Analysis (Including Missing Group):

     dp1 Score Group  Total_Clients  Avg_Arrears  % Mobile Available  % Email Available
            Decile 1             77     1,899.64               26.00              27.30
            Decile 2             83     3,698.15                8.40              19.30
            Decile 3             71     2,674.10               16.90              36.60
            Decile 4             80     2,168.44               21.20              50.00
            Decile 5             73     1,756.27               23.30              46.60
            Decile 6             78     1,050.95               50.00              30.80
            Decile 7             83       283.27               65.10              26.50
            Decile 8             69       198.60               60.90              31.90
            Decile 9             91       197.86               69.20              18.70
           Decile 10             61        55.25               62

## dp3 Score Group Analysis 

In [ ]:
import pandas as pd


df = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Modified/cleaned_connected_data_with_zones.csv")

# Step 1: Create dp3 Score Group with Deciles and a 'No Score' group
df['dp3 Score Group'] = pd.qcut(df[df['dp3 Score'].notnull()]['dp3 Score'], 10, labels=[f'Decile {i}' for i in range(1, 11)])
df['dp3 Score Group'] = df['dp3 Score Group'].astype(str)
df.loc[df['dp3 Score'].isnull(), 'dp3 Score Group'] = 'No Score (Unmatched)'

# Step 2: Clean Mobile and Email flags
df['Mobile Flag'] = df['Mobile Flag'].str.strip().str.upper()
df['Email Flag'] = df['Email Flag'].str.strip().str.upper()

# Step 3: Group by dp3 Score Group and summarize
summary = df.groupby('dp3 Score Group').agg(
    Total_Clients=('UID', 'count'),
    Avg_Arrears=('Arrears Balance', 'mean'),
    Mobile_Y=('Mobile Flag', lambda x: (x == 'Y').sum()),
    Email_Y=('Email Flag', lambda x: (x == 'Y').sum())
).reset_index()

# Step 4: Calculate percentage contactability
summary['% Mobile Available'] = (summary['Mobile_Y'] / summary['Total_Clients'] * 100).round(1)
summary['% Email Available'] = (summary['Email_Y'] / summary['Total_Clients'] * 100).round(1)

# Step 5: Reorder score groups with 'No Score' at the end
ordered_labels = [f'Decile {i}' for i in range(1, 11)] + ['No Score (Unmatched)']
summary['dp3 Score Group'] = pd.Categorical(summary['dp3 Score Group'], categories=ordered_labels, ordered=True)
summary = summary.sort_values('dp3 Score Group')

# Step 6: Select clean columns for output
summary = summary[['dp3 Score Group', 'Total_Clients', 'Avg_Arrears', '% Mobile Available', '% Email Available']]

# Step 7: Print or export result
pd.options.display.float_format = '{:,.2f}'.format
print("\n dp3 Score Group Analysis (Including Missing Group):\n")
print(summary.to_string(index=False))



📊 dp3 Score Group Analysis (Including Missing Group):

     dp3 Score Group  Total_Clients  Avg_Arrears  % Mobile Available  % Email Available
            Decile 1             75     4,186.19               10.70              16.00
            Decile 2             72     2,918.50                8.30              36.10
            Decile 3             74     2,539.52               16.20              39.20
            Decile 4             73     1,993.70               24.70              47.90
            Decile 5             73     1,368.24               41.10              35.60
            Decile 6             74       843.53               44.60              31.10
            Decile 7             80       400.01               61.30              30.00
            Decile 8             68        89.87               72.10              20.60
            Decile 9             79        55.14               73.40              15.20
           Decile 10             66        51.51               6

###  dp1 & dp3 Score Correlation to Client Data

#### 1. "How do dp1 and dp3 scores correlate with client data?"

**Score Metric** | **Client Data It Correlates With**
------------------|------------------------------------
dp1 Score          | As scores increase → arrears go down, contact goes up  
dp3 Score          | Same trend: higher scores = better contact & lower debt

 **This proves that the score is meaningful — it's not random.**  
It actually reflects contactability and debt behavior.

---

#### 2. "How can this help in debt management?"

Now that we know the scores are correlated with behavior, we can act on them.

| Insight | Action for Debt Teams |
|---------|------------------------|
| Deciles 9–10 → high score, low debt | Use softer outreach, automate contact |
| Deciles 1–2 → low score, high debt  | Use manual effort, involve tracing |
| No Score clients → uncertain risk  | Flag as unmatched, maybe verify or trace them separately |
| dp3 Score higher contact % than dp1 | dp3 might be more useful for targeting |

---

###  Summary Insight

I analyzed the dp1 and dp3 scores by breaking clients into deciles and comparing their contactability and arrears.  
I found that higher scores correlate with lower arrears and higher mobile availability — meaning the scores are good predictors of client behavior.  
These insights allow Connected Data’s clients to segment their customer base, prioritize contact strategies, and reduce costs by focusing effort where it’s needed most.



## dp2 

## Council Tax band

In [1]:
import pandas as pd


df = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Modified/cleaned_connected_data_with_zones.csv")

# Clean flags and tax band column
df['Mobile Flag'] = df['Mobile Flag'].str.strip().str.upper()
df['Email Flag'] = df['Email Flag'].str.strip().str.upper()
df['dp2 Council Tax Band'] = df['dp2 Council Tax Band'].str.strip().str.upper()

# Group by tax band
summary = df.groupby('dp2 Council Tax Band').agg(
    Total_Clients=('UID', 'count'),
    Avg_Arrears=('Arrears Balance', 'mean'),
    Avg_dp1_Score=('dp1 Score', 'mean'),
    Avg_dp3_Score=('dp3 Score', 'mean'),
    Mobile_Y=('Mobile Flag', lambda x: (x == 'Y').sum()),
    Email_Y=('Email Flag', lambda x: (x == 'Y').sum())
).reset_index()

# Calculate % contactable
summary['% Mobile Available'] = (summary['Mobile_Y'] / summary['Total_Clients'] * 100).round(1)
summary['% Email Available'] = (summary['Email_Y'] / summary['Total_Clients'] * 100).round(1)

# Final columns
summary = summary[['dp2 Council Tax Band', 'Total_Clients', 'Avg_Arrears',
                   'Avg_dp1_Score', 'Avg_dp3_Score', '% Mobile Available', '% Email Available']]

#  result
print("\n Council Tax Band Analysis:\n")
print(summary.to_string(index=False))



 Council Tax Band Analysis:

dp2 Council Tax Band  Total_Clients  Avg_Arrears  Avg_dp1_Score  Avg_dp3_Score  % Mobile Available  % Email Available
                   A            181   314.365854      76.440559     894.083333                61.3               22.7
                   B            181   484.035088      74.704545     876.157480                61.9               26.5
                   C            127   972.153153      65.250000     809.191011                48.8               29.9
                   D            145  1570.723077      55.869565     714.375000                33.8               31.7
                   E            153  2473.935252      43.000000     599.925620                16.3               37.9
                   F            127  2975.956522      34.096774     489.305882                 9.4               34.6


###  Council Tax Band Analysis

#### What This Analysis Shows

We grouped clients by **Council Tax Band (A to E)**, which reflects the property value — a proxy for a person’s financial strength.

Then I measured:
- Average arrears (how much they owe)  
- Average dp1 and dp3 Scores (credit trust)  
- % with Mobile  
- % with Email

---

### Inference:

| Tax Band | Property Value | Avg Arrears | Avg Scores | Contactability |
|----------|----------------|--------------|-------------|----------------|
| A–B      | Lower value     | Low arrears  | High scores | High mobile %  |
| D–E      | Higher value    | High arrears | Lower scores| Lower mobile % |

💡 **Insight**: Council Tax Band inversely correlates with affordability.

People in Band A have **lower debt**, **higher scores**, and **better contactability**.  
Band E clients may live in costlier homes, but carry **higher arrears** and **worse scores**.

---

### Debt Management

| Use Case                        | What to Do                                                           |
|--------------------------------|-----------------------------------------------------------------------|
| Prioritize soft contact for Band A/B | High chance of recovery, quick wins                        |
| Investigate Band D/E deeper    | Might appear wealthy but actually carry risky debt           |
| Combine with score + residency | Create multi-factor segmentation for smarter targeting        |



## Occupancy Style

In [ ]:
import pandas as pd


df = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Modified/cleaned_connected_data_with_zones.csv")

# Clean flags and Occupancy column
df['Mobile Flag'] = df['Mobile Flag'].str.strip().str.upper()
df['Email Flag'] = df['Email Flag'].str.strip().str.upper()
df['dp2 Occupancy Style'] = df['dp2 Occupancy Style'].str.strip().str.title()

# Group by Occupancy Style
summary = df.groupby('dp2 Occupancy Style').agg(
    Total_Clients=('UID', 'count'),
    Avg_Arrears=('Arrears Balance', 'mean'),
    Avg_dp1_Score=('dp1 Score', 'mean'),
    Avg_dp3_Score=('dp3 Score', 'mean'),
    Mobile_Y=('Mobile Flag', lambda x: (x == 'Y').sum()),
    Email_Y=('Email Flag', lambda x: (x == 'Y').sum())
).reset_index()

# Contactability %
summary['% Mobile Available'] = (summary['Mobile_Y'] / summary['Total_Clients'] * 100).round(1)
summary['% Email Available'] = (summary['Email_Y'] / summary['Total_Clients'] * 100).round(1)

# Final output
summary = summary[['dp2 Occupancy Style', 'Total_Clients', 'Avg_Arrears',
                   'Avg_dp1_Score', 'Avg_dp3_Score', '% Mobile Available', '% Email Available']]

# Print
pd.options.display.float_format = '{:,.2f}'.format
print("\n Occupancy Style Analysis:\n")
print(summary.to_string(index=False))



📊 Occupancy Style Analysis:

dp2 Occupancy Style  Total_Clients  Avg_Arrears  Avg_dp1_Score  Avg_dp3_Score  % Mobile Available  % Email Available
              Owner            411       796.48          69.12         836.92               50.60              31.60
     Private Renter            284     1,388.54          60.54         737.09               40.10              31.70
      Social Renter            261     2,295.07          46.19         613.85               27.20              23.80


### 1. What This Analysis Shows

We grouped clients by **Occupancy Style** (how they live):

- **Owner** = Stable homeowners  
- **Private Renter** = Likely to move, average risk  
- **Social Renter** = Often low income, supported housing

Then we looked at:
- Average arrears  
- Average scores (dp1 and dp3)  
- Contactability (mobile + email)

---

### 2. Key Insights

| Occupancy Style | Avg Arrears | Avg Score (dp1) | % Mobile | Risk Indicator                      |
|------------------|-------------|------------------|-----------|--------------------------------------|
| Owner            | £796       | 69.1             | 50.6%     | 🟢 Low risk, high score, stable contact |
| Private Renter   | £1,388     | 60.5             | 40.1%     | 🟠 Medium risk                         |
| Social Renter    | £2,295     | 46.2             | 27.2%     | 🔴 Highest risk – low score, high debt |

💡 **Conclusion**: Living status is a strong indicator of financial health and reachability.

---

### 3. Business Use Case

| Insight                         | Debt Strategy                                              |
|---------------------------------|------------------------------------------------------------|
| Owners → high score, low debt   | Prioritize with self-service or light touch               |
| Private Renters → mid-risk      | Use flexible recovery or SMS/Email first                 |
| Social Renters → high debt, low contact | Flag for sensitive treatment or deeper tracing required |

---



## Combined

#### What This Combined View Tells us:

I have grouped clients by:
- **dp2 Occupancy Style** (Owner, Private, Social)  
- **dp2 Council Tax Band** (A–E)  

Then for each group, we will see :
- Average Arrears  
- Average scores from dp1 and dp3  
- % with valid Mobile and Email

In [ ]:
import pandas as pd


df = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Modified/cleaned_connected_data_with_zones.csv")

# Step 1: Clean contact flags and dp2 fields
df['Mobile Flag'] = df['Mobile Flag'].str.strip().str.upper()
df['Email Flag'] = df['Email Flag'].str.strip().str.upper()
df['dp2 Council Tax Band'] = df['dp2 Council Tax Band'].str.strip().str.upper()
df['dp2 Occupancy Style'] = df['dp2 Occupancy Style'].str.strip().str.title()

# Step 2: Create combined profile with key fields
combined = df[['UID', 'dp1 Score', 'dp3 Score', 'dp2 Council Tax Band', 'dp2 Occupancy Style',
               'Arrears Balance', 'Mobile Flag', 'Email Flag']].copy()

# Step 3: Create contact flags
combined['Has Mobile'] = combined['Mobile Flag'] == 'Y'
combined['Has Email'] = combined['Email Flag'] == 'Y'

# Optional: Add risk bands from dp1/dp3 scores
combined['dp1 Risk Band'] = pd.qcut(combined['dp1 Score'], q=4, labels=['Very Low', 'Low', 'High', 'Very High'])
combined['dp3 Risk Band'] = pd.qcut(combined['dp3 Score'], q=4, labels=['Very Low', 'Low', 'High', 'Very High'])

# Step 4: Group by Occupancy + Council Tax Band for strategy view
summary = combined.groupby(['dp2 Occupancy Style', 'dp2 Council Tax Band']).agg(
    Total_Clients=('UID', 'count'),
    Avg_Arrears=('Arrears Balance', 'mean'),
    Avg_dp1_Score=('dp1 Score', 'mean'),
    Avg_dp3_Score=('dp3 Score', 'mean'),
    Pct_With_Mobile=('Has Mobile', lambda x: (x.sum() / len(x) * 100).round(1)),
    Pct_With_Email=('Has Email', lambda x: (x.sum() / len(x) * 100).round(1))
).reset_index()

# Step 5: Display or export
pd.options.display.float_format = '{:,.2f}'.format
print("\n📊 Combined Strategy View by Client Profile:\n")
print(summary.to_string(index=False))



📊 Combined Strategy View by Client Profile:

dp2 Occupancy Style dp2 Council Tax Band  Total_Clients  Avg_Arrears  Avg_dp1_Score  Avg_dp3_Score  Pct_With_Mobile  Pct_With_Email
              Owner                    A            103       192.39          78.72         932.35            64.10           23.30
              Owner                    B             89       350.55          73.61         904.53            62.90           29.20
              Owner                    C             53       698.21          67.20         856.50            60.40           28.30
              Owner                    D             54     1,397.08          58.93         735.86            33.30           33.30
              Owner                    E             38     1,815.53          56.94         716.29            23.70           50.00
              Owner                    F             31     2,190.52          49.86         591.45            12.90           54.80
     Private Renter           

###  Debt Strategy:

**Profile Example** | **Interpretation** | **Action**
---|---|---
Owner + Band A | Low arrears, high scores, high mobile rate | Priority for fast recovery via mobile  
Private Renter + Band D | Medium arrears, mid score, low mobile | Medium strategy with reminders/email  
Social Renter + Band E | High arrears, low score, low mobile/email | High risk, consider tracing or external

---

###  Combined Strategy View

Now we have a **combined strategy view** using metrics from **all 3 data partners**.

By cross-tabulating **dp1 and dp3 scores** with **dp2’s property and residency info**,  
I created actionable client segments that support **prioritization, tracing, and outreach decisions**.

 This lets debt teams **focus on clients most likely to pay**, while **minimizing cost and risk**.



In [14]:
print("Missing Council Tax Band:", df['dp2 Council Tax Band'].isnull().sum())
print("Missing Occupancy Style:", df['dp2 Occupancy Style'].isnull().sum())


Missing Council Tax Band: 86
Missing Occupancy Style: 44


###  About the Missing Data?

dp2 Council Tax Band is missing for **86 clients**  
dp2 Occupancy Style is missing for **44 clients**

That’s:
- 8.6% missing for tax band
- 4.4% missing for occupancy style

We still have over **900 valid clients** with full dp2 intel — that’s totally fine for meaningful analysis.

If we don’t want to exclude them — we can treat them as an **'Unknown' group for risk review**.

In production, these clients could be flagged for enrichment or deferred action.

---



##  Further Analysis

#### Method
1. **Segment Risk Score** – Turns our existing table into a ranked scoring sheet. Easy to sort, compare, explain.  
2. **Action Tier** – Converts those scores into a simple, 3-level strategy. Helps the business act on analysis.

---

Now I have evaluated all **3 data partners**,  
I combined their signals into a custom **risk score**, and then translated that score into a **3-tier strategy**.

This lets collection teams instantly see:
- who to contact first,
- who needs verification,
- and who to de-prioritize.





###  How the Score is Built

We can build it by:

**1. Create a total segment score using:**
- High arrears = +risk
- Low dp1 & dp3 = +risk
- Low contactability = +risk

**2. Add a final column called Action Tier based on score ranges**

---

###  Risk Score Components Table

| Metric Used       | What it Does                | In Risk Score Logic                                   |
|-------------------|-----------------------------|--------------------------------------------------------|
| Avg_Arrears       | High arrears = more risk    | + Avg_Arrears / 100                                   |
| dp1 Score         | Lower score = more risk     | + (100 - dp1_score)                                   |
| dp3 Score         | Lower score = more risk     | + (100 - dp3_score) / 10                              |
| % Mobile / Email  | Lower = harder to reach     | + (100 - Pct_Mobile)/2 + (100 - Pct_Email)/3          |

---

###  Action Tier Legend:

| Tier        | Action                                                  |
|-------------|----------------------------------------------------------|
| Tier 1 – Low    | Low debt, high score, reachable → Light reminders, SMS, digital |
| Tier 2 – Medium | Mixed scores or medium contact → Manual call, strategy check  |
| Tier 3 – High   | High debt, unreachable, low score → Escalation, tracing, deprioritize |

---

###  Typical Range – What It Means

| Range         | Risk Level           |
|---------------|----------------------|
| Below 70      | Low Risk (Tier 1)    |
| 70–100        | Medium Risk (Tier 2) |
| 100–200+      | High Risk (Tier 3)   |

---

Instead of manually looking at 6 different columns (arrears, dp1, dp3, mobile, email...),  
now we have **1 final number** that tells you how risky a group is.

 The Segment Risk Score is like a **combined health check** for every client group.



In [ ]:
import pandas as pd

#  dataset
df = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Modified/cleaned_connected_data_with_zones.csv")

# Step 1: Clean flags and dp2 fields
df['Mobile Flag'] = df['Mobile Flag'].str.strip().str.upper()
df['Email Flag'] = df['Email Flag'].str.strip().str.upper()
df['dp2 Council Tax Band'] = df['dp2 Council Tax Band'].str.strip().str.upper()
df['dp2 Occupancy Style'] = df['dp2 Occupancy Style'].str.strip().str.title()

# Step 2: Create flags for contact availability
df['Has Mobile'] = df['Mobile Flag'] == 'Y'
df['Has Email'] = df['Email Flag'] == 'Y'

# Step 3: Group by dp2 segments
summary = df.groupby(['dp2 Occupancy Style', 'dp2 Council Tax Band']).agg(
    Total_Clients=('UID', 'count'),
    Avg_Arrears=('Arrears Balance', 'mean'),
    Avg_dp1_Score=('dp1 Score', 'mean'),
    Avg_dp3_Score=('dp3 Score', 'mean'),
    Mobile_Y=('Has Mobile', 'sum'),
    Email_Y=('Has Email', 'sum')
).reset_index()

# Step 4: Calculate % contactable
summary['Pct_Mobile'] = (summary['Mobile_Y'] / summary['Total_Clients'] * 100).round(1)
summary['Pct_Email'] = (summary['Email_Y'] / summary['Total_Clients'] * 100).round(1)

# Step 5: Calculate total Segment Risk Score
summary['Segment Risk Score'] = (
    summary['Avg_Arrears'] / 100 +                            # More arrears = more risk
    (100 - summary['Avg_dp1_Score']) +                        # Lower dp1 = more risk
    (100 - summary['Avg_dp3_Score']) / 10 +                   # Lighter weight for dp3
    (100 - summary['Pct_Mobile']) / 2 +                       # Less contactability = more risk
    (100 - summary['Pct_Email']) / 3                          # Less contactability = more risk
).round(1)

# Step 6: Assign Action Tier
def assign_action_tier(score):
    if score >= 100:
        return "Tier 3 - High Risk"
    elif score >= 70:
        return "Tier 2 - Medium Risk"
    else:
        return "Tier 1 - Low Risk"

summary['Action Tier'] = summary['Segment Risk Score'].apply(assign_action_tier)

# Step 7: Final clean output
final_strategy = summary[['dp2 Occupancy Style', 'dp2 Council Tax Band', 'Total_Clients',
                          'Avg_Arrears', 'Avg_dp1_Score', 'Avg_dp3_Score',
                          'Pct_Mobile', 'Pct_Email', 'Segment Risk Score', 'Action Tier']]

# Display it in a readable format
pd.options.display.float_format = '{:,.2f}'.format
print("\n📊 OLD Segmented Strategy Table:\n")
print(final_strategy.to_string(index=False))



📊 OLD Segmented Strategy Table:

dp2 Occupancy Style dp2 Council Tax Band  Total_Clients  Avg_Arrears  Avg_dp1_Score  Avg_dp3_Score  Pct_Mobile  Pct_Email  Segment Risk Score          Action Tier
              Owner                    A            103       192.39          78.72         932.35       64.10      23.30              -16.50    Tier 1 - Low Risk
              Owner                    B             89       350.55          73.61         904.53       62.90      29.20               -8.40    Tier 1 - Low Risk
              Owner                    C             53       698.21          67.20         856.50       60.40      28.30                7.80    Tier 1 - Low Risk
              Owner                    D             54     1,397.08          58.93         735.86       33.30      33.30               47.00    Tier 1 - Low Risk
              Owner                    E             38     1,815.53          56.94         716.29       23.70      50.00               54.40    Tier 1

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Modified/cleaned_connected_data_with_zones.csv")

# Step 1: Clean flags and dp2 fields
df['Mobile Flag'] = df['Mobile Flag'].str.strip().str.upper()
df['Email Flag'] = df['Email Flag'].str.strip().str.upper()
df['dp2 Council Tax Band'] = df['dp2 Council Tax Band'].str.strip().str.upper()
df['dp2 Occupancy Style'] = df['dp2 Occupancy Style'].str.strip().str.title()

# Step 2: Create flags for contact availability
df['Has Mobile'] = df['Mobile Flag'] == 'Y'
df['Has Email'] = df['Email Flag'] == 'Y'

# Step 3: Group by dp2 segments
summary = df.groupby(['dp2 Occupancy Style', 'dp2 Council Tax Band']).agg(
    Total_Clients=('UID', 'count'),
    Avg_Arrears=('Arrears Balance', 'mean'),
    Avg_dp1_Score=('dp1 Score', 'mean'),
    Avg_dp3_Score=('dp3 Score', 'mean'),
    Mobile_Y=('Has Mobile', 'sum'),
    Email_Y=('Has Email', 'sum')
).reset_index()

# Step 4: Calculate % contactable
summary['Pct_Mobile'] = (summary['Mobile_Y'] / summary['Total_Clients'] * 100).round(1)
summary['Pct_Email'] = (summary['Email_Y'] / summary['Total_Clients'] * 100).round(1)

# Step 5: Scale dp3 score and calculate Segment Risk Score
summary['Scaled_dp3_Score'] = summary['Avg_dp3_Score'] / 10  # Convert to 0–100 scale

summary['Segment_Risk_Score'] = (
    (summary['Avg_Arrears'] / 100) +
    (100 - summary['Avg_dp1_Score']) +
    (100 - summary['Scaled_dp3_Score']) +
    ((100 - summary['Pct_Mobile']) / 2) +
    ((100 - summary['Pct_Email']) / 3)
).round(1)

# Step 6: Assign Action Tier
def assign_action_tier(score):
    if score >= 100:
        return "Tier 3 - High Risk"
    elif score >= 70:
        return "Tier 2 - Medium Risk"
    else:
        return "Tier 1 - Low Risk"

summary['Action_Tier'] = summary['Segment_Risk_Score'].apply(assign_action_tier)

# Step 7:  output
final_strategy = summary[['dp2 Occupancy Style', 'dp2 Council Tax Band', 'Total_Clients',
                          'Avg_Arrears', 'Avg_dp1_Score', 'Avg_dp3_Score',
                          'Pct_Mobile', 'Pct_Email', 'Segment_Risk_Score', 'Action_Tier']]

# Display
pd.options.display.float_format = '{:,.2f}'.format
print("\n Final Segmented Strategy Table:\n")
print(final_strategy.to_string(index=False))



📊 Final Segmented Strategy Table:

dp2 Occupancy Style dp2 Council Tax Band  Total_Clients  Avg_Arrears  Avg_dp1_Score  Avg_dp3_Score  Pct_Mobile  Pct_Email  Segment_Risk_Score          Action_Tier
              Owner                    A            103       192.39          78.72         932.35       64.10      23.30               73.50 Tier 2 - Medium Risk
              Owner                    B             89       350.55          73.61         904.53       62.90      29.20               81.60 Tier 2 - Medium Risk
              Owner                    C             53       698.21          67.20         856.50       60.40      28.30               97.80 Tier 2 - Medium Risk
              Owner                    D             54     1,397.08          58.93         735.86       33.30      33.30              137.00   Tier 3 - High Risk
              Owner                    E             38     1,815.53          56.94         716.29       23.70      50.00              144.40   Tier 

###  Segment Risk Score Formula Breakdown

```
Avg_Arrears / 100         → More debt = more risk         ↑ Increases score  
100 - dp1_score           → Lower credit score = risk     ↑ Increases score  
(100 - dp3_score) / 10    → Same, but lighter weight      ↑ Slight increase  
(100 - % Mobile) / 2      → Harder to contact = risk      ↑ Increase score  
(100 - % Email) / 3       → Same                          ↑ Small increase  
```

---

###  Constraints for Tier 1 – Low Risk

| Metric         | Value Needed                  |
|----------------|-------------------------------|
| Avg Arrears    | Very low (under £500 ideally) |
| dp1 Score      | High (75–100)                 |
| dp3 Score      | High (900+)                   |
| % Mobile       | 60–70%+                        |
| % Email        | 40–60%+                        |

---

###  Maximum Possible Score (Worst-case client group)

| Component         | Max Risk | Explanation                       |
|------------------|----------|-----------------------------------|
| Arrears           | 30–40    | If average debt is ~£3000–4000    |
| 100 - dp1         | 100      | dp1 Score = 0                     |
| 100 - dp3/10      | 100      | dp3 Score = 0                     |
| (100 - Mobile)/2  | 50       | Pct_Mobile = 0%                   |
| (100 - Email)/3   | 33.3     | Pct_Email = 0%                    |

 **Total Max Score ≈ 320–330 (Very Rare)**

---

###  Minimum Possible Score (Best-case client group)

| Component         | Min Risk | Explanation                      |
|------------------|----------|----------------------------------|
| Arrears           | 0.5–2.0  | Very low debt (£50–£200)         |
| 100 - dp1         | 0        | dp1 Score = 100                  |
| 100 - dp3/10      | 0        | dp3 Score = 1000                 |
| (100 - Mobile)/2  | 0        | Pct_Mobile = 100%                |
| (100 - Email)/3   | 0        | Pct_Email = 100%                 |

 **Total Min Score ≈ 1 to 3**



## Merging scores and tiers adding a new dimension

In [ ]:
import pandas as pd


final_strategy = pd.DataFrame({
    'dp2 Occupancy Style': ['Owner', 'Owner', 'Owner', 'Owner', 'Owner', 'Owner',
                            'Private Renter', 'Private Renter', 'Private Renter', 'Private Renter', 'Private Renter', 'Private Renter',
                            'Social Renter', 'Social Renter', 'Social Renter', 'Social Renter', 'Social Renter', 'Social Renter'],
    'dp2 Council Tax Band': ['A', 'B', 'C', 'D', 'E', 'F',
                             'A', 'B', 'C', 'D', 'E', 'F',
                             'A', 'B', 'C', 'D', 'E', 'F'],
    'Segment_Risk_Score': [73.5, 81.6, 97.8, 137.0, 144.4, 171.5,
                           73.0, 86.8, 108.1, 136.7, 192.1, 213.1,
                           125.7, 99.0, 144.7, 160.7, 212.6, 238.0],
    'Action_Tier': ['Tier 2 – Medium Risk', 'Tier 2 – Medium Risk', 'Tier 2 – Medium Risk', 'Tier 3 – High Risk', 'Tier 3 – High Risk', 'Tier 3 – High Risk',
                    'Tier 2 – Medium Risk', 'Tier 2 – Medium Risk', 'Tier 3 – High Risk', 'Tier 3 – High Risk', 'Tier 3 – High Risk', 'Tier 3 – High Risk',
                    'Tier 3 – High Risk', 'Tier 2 – Medium Risk', 'Tier 3 – High Risk', 'Tier 3 – High Risk', 'Tier 3 – High Risk', 'Tier 3 – High Risk']
})

# Save this final group score table
final_strategy.to_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Result_Tables/final_segmented_strategy.csv", index=False)

print(" Saved final_segmented_strategy.csv successfully!")


 Saved final_segmented_strategy.csv successfully!


In [ ]:
import pandas as pd

# dataset
clients = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Modified/cleaned_connected_data_with_zones.csv")


group_scores = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Result_Tables/final_segmented_strategy.csv")

#  group_scores
group_scores = group_scores[['dp2 Occupancy Style', 'dp2 Council Tax Band', 'Segment_Risk_Score', 'Action_Tier']]

# Merging risk scores 
clients_with_risk = clients.merge(group_scores, on=['dp2 Occupancy Style', 'dp2 Council Tax Band'], how='left')


clients_with_risk.to_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Modified/clients_with_segment_risk.csv", index=False)

print(" Successfully added Segment Risk Score and Action Tier to all clients!")


 Successfully added Segment Risk Score and Action Tier to all clients!
